We have esp-idf checked out and installed.

https://docs.espressif.com/projects/esp-idf/en/latest/get-started/index.html#get-started-get-esp-idf

conda deactivate

Go into directory:  extrepositories/esp-idf and execute ". ./export.sh"

Then go into extrepositories/micropython-hack/ports/esp32/  and do: "make"

(If necessary, go into mpy-cross and make)

If make fails it will tell you which checkout you are allowed
Go into extrepositories/esp-i2s
eg: "git checkout 310beae373446ceb9a4ad9b36b5428d7fdf2705f"
Then: git submodule update --init --recursive

Then ./install.sh
. ./export.sh

---
current scheme applied the patch https://patch-diff.githubusercontent.com/raw/micropython/micropython/pull/4471.patch  (not trying to do as a checkout!)



In [2]:
%esptool esp32 /home/julian/extrepositories/micropython-hack/ports/esp32/build-GENERIC/firmware.bin


Executing:
  esptool.py --port /dev/ttyUSB0 --chip esp32 write_flash -z 0x1000 /home/julian/extrepositories/micropython-hack/ports/esp32/build-GENERIC/firmware.bin

esptool.py v2.8
Serial port /dev/ttyUSB0
Connecting....
[Press the PRG button now if required]
Chip is ESP32D0WDQ6 (revision 1)
Features: WiFi, BT, Dual Core, 240MHz, VRef calibration in efuse, Coding Scheme None
Crystal is 26MHz
MAC: 30:ae:a4:fe:a1:5c
Uploading stub...
Running stub...
Stub running...
Configuring flash size...
Auto-detected Flash size: 4MB
Compressed 1412256 bytes to 899806...
Wrote 1412256 bytes (899806 compressed) at 0x00001000 in 80.1 seconds (effective 141.0 kbit/s)...
Hash of data verified.

Leaving...
Hard resetting via RTS pin...


In [3]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [97]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [44]:
%sendtofile --binary --source /home/julian/Music/taunt-16k-16bits-mono-12db.wav

Sent 131226 bytes in 4375 chunks to taunt-16k-16bits-mono-12db.wav.


In [101]:
%sendtofile --binary --source /home/julian/data/moo2g.wav

Sent 53174 bytes in 1773 chunks to moo2g.wav.


In [105]:
from machine import I2S
from machine import Pin

audio_out = I2S(I2S.NUM1, bck=Pin(12), ws=Pin(13), sdout=Pin(14), 
              standard=I2S.PHILIPS, mode=I2S.MASTER_TX,
              dataformat=I2S.B16, channelformat=I2S.ONLY_RIGHT,
              samplerate=16000,
              dmacount=8, dmalen=512)

s = open('moo2g.wav', 'rb')
s.seek(44) # advance to first byte of Data section in WAV file

# continuously read audio samples from the WAV file 
# and write them to an I2S DAC
while True:
    try:
        audio_samples = bytearray(s.read(1024))
        numwritten = 0
        # end of WAV file?    
        if len(audio_samples) == 0:
            s.seek(44) # advance to first byte of Data section
        else:
            # loop until samples can be written to DMA
            while numwritten == 0:
                # return immediately when no DMA buffer is available (timeout=0)
                numwritten = audio_out.write(audio_samples, timeout=0)
                
                # await - allow other coros to run   
    except KeyboardInterrupt:  
        s.close()
        audio_out.deinit()
        break

.

*** Sending Ctrl-C



In [86]:

audio_out = I2S(I2S.NUM1, bck=bck_pin, ws=ws_pin, sdout=sdout_pin, 
              standard=I2S.PHILIPS, mode=I2S.MASTER_TX,
              dataformat=I2S.B16, channelformat=I2S.ONLY_RIGHT,
              samplerate=SAMPLES_PER_SECOND,
              dmacount=8, dmalen=512)


In [87]:
audio_out.write(audio_samples, timeout=0)


In [104]:
audio_out.deinit()


In [88]:
samples = bytearray(1024)                                 # bytearray containing audio samples to transmit
for i in range(1024):
    samples[i] = 0x01
import time
while True:
    audio_out.write(samples, timeout=0)
    time.sleep_ms(500)


........................................
**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)


**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)

